In [1]:
import pyscf
import pyscf.tools

from orbitalpartitioning import *

/home/arnabbachhar/anaconda3/lib/python3.10/site-packages/pyscf/dft/libxc.py:772: UserWarning: Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, the same to the B3LYP functional in Gaussian and ORCA (issue 1480). To restore the VWN5 definition, you can put the setting "B3LYP_WITH_VWN5 = True" in pyscf_conf.py
  warnings.warn('Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, '


In [2]:
molecule = """
 Cr 0.82627800 -1.30446200 -0.96524300
 Cr -0.82625500 1.30449100 0.96521400
 O 0.00001100 0.00001500 -0.00001400
 N 2.71379100 -0.58517300 -0.32156700
 H 3.52017600 -0.87998300 -0.88101600
 H 2.98302700 -0.84252100 0.62848200
 H 2.75216700 0.43162900 -0.34666000
 N 0.90611400 -0.01191500 -2.64373800
 H 0.15767300 0.67336300 -2.59732700
 H 0.79520000 -0.45289000 -3.56150100
 H 1.76766300 0.52845600 -2.74461100
 N 0.74644200 -2.59700900 0.71325300
 H 0.86887400 -3.59499400 0.51496700
 H -0.15371900 -2.54555600 1.18538900
 H 1.44183900 -2.41670700 1.43797600
 N -1.06123500 -2.02375100 -1.60891800
 H -1.51982700 -2.67507800 -0.96956400
 H -1.07760500 -2.52098500 -2.50431600
 H -1.71705200 -1.25706600 -1.73635500
 N 1.78821400 -2.82312800 -2.08895700
 H 2.20721200 -2.51946500 -2.97231100
 H 1.19687500 -3.61071400 -2.36865500
 H 2.56853800 -3.28036700 -1.60828700
 N -2.71376800 0.58520300 0.32153900
 H -2.72742600 -0.43151600 0.28926300
 H -3.01495100 0.88961900 -0.60504900
 H -3.51124000 0.82996900 0.91689300
 N -0.90609100 0.01194400 2.64371000
 H -0.86251400 0.45862400 3.56440800
 H -0.11862500 -0.63055500 2.63771800
 H -1.73984000 -0.57546100 2.70589400
 N 1.06125800 2.02378000 1.60888900
 H 1.08580100 2.47872200 2.52630400
 H 1.50134100 2.71049000 0.99382500
 H 1.72819800 1.26071800 1.68834600
 N -0.74641900 2.59703800 -0.71328200
 H -0.92607700 3.58837700 -0.52509900
 H -1.40673500 2.37764300 -1.45939400
 H 0.17080800 2.59240800 -1.15542700
 N -1.78819100 2.82315700 2.08892800
 H -1.19783000 3.61531400 2.35756800
 H -2.19798000 2.52398300 2.97800300
 H -2.57540900 3.27269300 1.61233200
"""

basis = "def2-svp"
pymol = pyscf.gto.Mole(
        atom    =   molecule,
        symmetry=   True,
        spin    =   6, # number of unpaired electrons
        charge  =   4,
        basis   =   basis)


pymol.build()
print("symmetry: ",pymol.topgroup)
# mf = pyscf.scf.UHF(pymol).x2c()
mf = pyscf.scf.ROHF(pymol)
mf.verbose = 4
mf.conv_tol = 1e-8
mf.conv_tol_grad = 1e-5
mf.chkfile = "scf.fchk"
mf.init_guess = "sad"

mf.run(max_cycle=200)

print(" Hartree-Fock Energy: %12.8f" % mf.e_tot)
# mf.analyze()

# Get data
F = mf.get_fock()
C = mf.mo_coeff
S = mf.get_ovlp()
ndocc = mf.nelec[1]
nsing = mf.nelec[0] - ndocc
nvirt = mf.mol.nao - ndocc - nsing

# Just use alpha orbitals
Cdocc = mf.mo_coeff[:,0:ndocc]
Csing = mf.mo_coeff[:,ndocc:ndocc+nsing]
Cvirt = mf.mo_coeff[:,ndocc+nsing:ndocc+nsing+nvirt]

nbas = Cdocc.shape[0]

symmetry:  C1


******** <class 'pyscf.scf.rohf.ROHF'> ********
method = ROHF-RHF
initial guess = sad
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
SCF conv_tol = 1e-08
SCF conv_tol_grad = 1e-05
SCF max_cycles = 200
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = scf.fchk
max_memory 4000 MB (current use 132 MB)
num. doubly occ = 73  num. singly occ = 6


init E= -2722.30649096235
  HOMO = -0.180893429449352  LUMO = -0.0912350732357864
cycle= 1 E= -2721.10285451241  delta_E=  1.2  |g|= 1.19  |ddm|= 4.33
  HOMO = -0.48431336541647  LUMO = -0.390433229330149
cycle= 2 E= -2721.51059453132  delta_E= -0.408  |g|= 0.767  |ddm|= 1.34
  HOMO = -0.597263143223493  LUMO = -0.394693000098286
cycle= 3 E= -2721.6460417894  delta_E= -0.135  |g|= 0.273  |ddm|= 0.643
  HOMO = -0.658985859320278  LUMO = -0.39026425175723
cycle= 4 E= -2721.66337406942  delta_E= -0.0173  |g|= 0.102  |ddm|= 0.278
  HOMO = -0.629457238654777  LUMO = -0.388999756567965
cycle= 5 E= -2721.66747378762  delta_E= -0.0041  |g|= 0.0297  |ddm|= 0.123
  HOMO = -0.638725503835443  LUMO = -0.388775332210024
cycle= 6 E= -2721.66781991664  delta_E= -0.000346  |g|= 0.00503  |ddm|= 0.0409
  HOMO = -0.638891031522767  LUMO = -0.38879170727771
cycle= 7 E= -2721.66783324035  delta_E= -1.33e-05  |g|= 0.00265  |ddm|= 0.015
  HOMO = -0.639229284649006  LUMO = -0.388820079206928
cycle= 8 E= -2721

# Define Fragments by AOs

In [3]:
# Find AO's corresponding to atoms
full = []
frag1 = []
frag2 = []
frag3 = []
for ao_idx,ao in enumerate(mf.mol.ao_labels(fmt=False)):
    if ao[0] == 0:
        if ao[2] in ("3d","4d"):
            frag1.append(ao_idx)
            full.append(ao_idx)
    elif ao[0] == 2:
        if ao[2] in ( "2p","3p"):
            frag2.append(ao_idx)
            full.append(ao_idx)
    elif ao[0] == 1:
        if ao[2] in ("3d", "4d"):
            frag3.append(ao_idx)
            full.append(ao_idx)


frags = [frag1, frag2, frag3]
print(frags)


[[14, 15, 16, 17, 18, 19, 20, 21, 22, 23], [65, 66, 67, 68, 69, 70], [45, 46, 47, 48, 49, 50, 51, 52, 53, 54]]


# Define Projectors
We can choose to project onto the non-orthogonal AOs, or onto the symmetrically orthogonalized AOs.

In [4]:
# Define projectors
X = np.eye(nbas) 
X = scipy.linalg.sqrtm(S)
Pfull = X[:,full]  # non-orthogonal
Pf = []
for f in frags:
    Pf.append(X[:,f])


# Project MOs onto all fragments
For each orbital block (Docc, Sing, Virt), project each subspace onto the full list of fragment AOs. This will determine our full active space.

In [5]:
(Oact, Sact, Vact), (Cenv, Cerr, _) = svd_subspace_partitioning((Cdocc, Csing, Cvirt), Pfull, S)
assert(Cerr.shape[1] == 0)
Cact = np.hstack((Oact,Vact))

 Partition  366 orbitals into a total of   26 orbitals
            Index   Sing. Val. Space       
                0   1.02099786            2*
                1   1.01920210            2*
                2   1.01230121            2*
                3   0.99139989            2*
                4   0.98172394            2*
                5   0.97853079            2*
                6   0.97056796            1*
                7   0.97046336            1*
                8   0.95191153            1*
                9   0.95183396            1*
               10   0.95045603            1*
               11   0.95027156            1*
               12   0.94507022            2*
               13   0.80017305            2*
               14   0.79720505            2*
               15   0.78597831            0*
               16   0.76414031            0*
               17   0.76400905            0*
               18   0.74105114            2*
               19   0.73777442            2*
 

# Split active space into fragments

In [6]:
# Project active orbitals onto fragments
init_fspace = []
clusters = []
Cfrags = []
orb_index = 1



for fi,f in enumerate(frags):
    print()
    print(" Fragment: ", f)
    (Of, Sf, Vf), (_, _, _) = svd_subspace_partitioning((Oact, Sact, Vact), Pf[fi], S)
    Cfrags.append(np.hstack((Of, Sf, Vf)))
    ndocc_f = Of.shape[1]
    init_fspace.append((ndocc_f+Sf.shape[1], ndocc_f))
    nmof = Of.shape[1] + Sf.shape[1] + Vf.shape[1]
    clusters.append(list(range(orb_index, orb_index+nmof)))
    orb_index += nmof



# Orthogonalize Fragment orbitals
Cfrags = sym_ortho(Cfrags, S)

Cact = np.hstack(Cfrags)

# Write Molden files for visualization
pyscf.tools.molden.from_mo(mf.mol, "Pfull.molden", Pfull)
pyscf.tools.molden.from_mo(mf.mol, "Cact.molden", Cact)
pyscf.tools.molden.from_mo(mf.mol, "Cenv.molden", Cenv)
for i in range(len(frags)):
    pyscf.tools.molden.from_mo(mf.mol, "Cfrag%i.molden"%i, Cfrags[i])

print(" init_fspace: ", init_fspace)
print(" clusters   : ", clusters)


# data["Cact"] = Cact
# data["init_fspace"] = init_fspace 
# data["clusters"] = clusters 
# with open('data_CrOCr.pickle', 'wb') as handle:
#     pickle.dump(data, handle)



 Fragment:  [14, 15, 16, 17, 18, 19, 20, 21, 22, 23]
 Partition   26 orbitals into a total of   10 orbitals
            Index   Sing. Val. Space       
                0   0.98007465            2*
                1   0.95989527            1*
                2   0.95973770            1*
                3   0.95782442            2*
                4   0.95185917            1*
                5   0.72128293            2*
                6   0.66266350            2*
                7   0.65977568            2*
                8   0.41649385            0*
                9   0.37877058            0*
               10   0.37632206            0
               11   0.34825152            0
               12   0.22437898            0
               13   0.05495248            0
               14   0.05419877            0
               15   0.04890189            1
               16   0.04865765            1
               17   0.04258179            2
               18   0.02320212            2
 

# Make Integrals

In [7]:
print(Cenv.shape)
print(Cact.shape)
d1_embed = 2 * Cenv @ Cenv.T

h0 = pyscf.gto.mole.energy_nuc(mf.mol)
h  = pyscf.scf.hf.get_hcore(mf.mol)
j, k = pyscf.scf.hf.get_jk(mf.mol, d1_embed, hermi=1)

print(h.shape)

(366, 66)
(366, 26)
(366, 366)


In [8]:
h0 += np.trace(d1_embed @ ( h + .5*j - .25*k))

h = Cact.T @ h @ Cact;
j = Cact.T @ j @ Cact;
k = Cact.T @ k @ Cact;

In [9]:
nact = h.shape[0]

h2 = pyscf.ao2mo.kernel(pymol, Cact, aosym="s4", compact=False)
h2.shape = (nact, nact, nact, nact)

In [10]:
# The use of d1_embed only really makes sense if it has zero electrons in the
# active space. Let's warn the user if that's not true

S = pymol.intor("int1e_ovlp_sph")
n_act = np.trace(S @ d1_embed @ S @ Cact @ Cact.T)
if abs(n_act) > 1e-8 == False:
    print(n_act)
    error(" I found embedded electrons in the active space?!")

h1 = h + j - .5*k;


In [11]:
np.save("ints_h0", h0)
np.save("ints_h1", h1)
np.save("ints_h2", h2)
np.save("mo_coeffs", Cact)
np.save("overlap_mat", S)

In [13]:
import numpy as np
Ccmf = np.load("Ccmf_26_cr2.npy")
pyscf.tools.molden.from_mo(mf.mol, "Ccmf_26_cr2.molden", Ccmf)
